# Data Preprocessing

In [1]:
import pandas as pd
import numpy as np

### Chargement des données True et Fake

In [2]:
# Chargement des données csv 
df_true = pd.read_csv('/home/fadilatou/PROJETS/rag_fake_news/data/True.csv')
df_fake = pd.read_csv('/home/fadilatou/PROJETS/rag_fake_news/data/Fake.csv')

# Affichage
print(df_true.head())
print(df_fake.head())

# Création de la colonne label
df_true["label"] = "True"
df_fake["label"] = "Fake"

# Fusionner les deux datasets
df = pd.concat([df_true, df_fake], ignore_index=True)

# Afficher les premières lignes
print(df.head())


                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   
2  December 31, 2017   
3  December 30, 2017   
4  December 29, 2017   
                                               title  \
0   Donald Trump Sends Out Embarrassing Ne

###  Exploration de données 

In [3]:
# Les dimensions de chaque df
print(df.shape)

# Les types de données
print(f" types de données:",df.dtypes)

# Le résumer statistique des colonnes
print(df.describe())

# Les valeurs de la colonne labels
print(f"Valeur du label:",df['label'].value_counts())


(44898, 5)
 types de données: title      object
text       object
subject    object
date       object
label      object
dtype: object
                                                    title   text  \
count                                               44898  44898   
unique                                              38729  38646   
top     Factbox: Trump fills top jobs for his administ...          
freq                                                   14    627   

             subject                date  label  
count          44898               44898  44898  
unique             8                2397      2  
top     politicsNews  December 20, 2017    Fake  
freq           11272                 182  23481  
Valeur du label: label
Fake    23481
True    21417
Name: count, dtype: int64


### Nettoyage du Dataset

In [4]:
# # # La somme des valeurs manquantes
print(f"Les valeurs manquantes :", df.isnull().sum())

# # Détection des doublons
print(f"Nombre de doublons:", df.duplicated().sum())

# Supprimer les doublons
df = df.drop_duplicates(subset=['title', 'text'])

# Vérification
print(".csv après nettoyage :")
print("Doublons restants :", df.duplicated(subset=['title', 'text']).sum())


Les valeurs manquantes : title      0
text       0
subject    0
date       0
label      0
dtype: int64
Nombre de doublons: 209
.csv après nettoyage :
Doublons restants : 0


In [5]:
# Conversion de la colonne date en datetime
df["date"] = pd.to_datetime(df["date"], errors = 'coerce')
print(df["date"].head())

0   2017-12-31
1   2017-12-29
2   2017-12-31
3   2017-12-30
4   2017-12-29
Name: date, dtype: datetime64[ns]


In [6]:
# # Sauvegarde du dataset propre 
# df.to_csv("/home/fadilatou/PROJETS/rag_fake_news/data/news_clean.csv", index=False)

### Application des fonction de nettoyage sur les colonnes text et title

In [7]:
from test_cleaning import clean_text_pipeline

# Tester sur les 5 premières lignes pour vérifier la rapidité
df["text"] = df["text"].apply(clean_text_pipeline)
df["title"] = df["title"].apply(clean_text_pipeline)

fake news politics


### Chunkings : Découpage des données de la colonne text

In [8]:
def chunk_text(text, chunk_size=200, overlap=50):
    """
    Découpe un texte en plusieurs morceaux (chunks) avec un chevauchement optionnel.

    text : str → le texte à découper
    chunk_size : int → nombre de mots par chunk
    overlap : int → nombre de mots qui se chevauchent entre deux chunks
    """
    if not isinstance(text, str) or not text.strip():
        return []  # Retourne une liste vide si le texte est vide ou non valide
    
    words = text.split()
    chunks = []

    # Taille du pas entre deux chunks
    step = chunk_size - overlap
    for i in range(0, len(words), step):
        chunk = words[i:i + chunk_size]
        chunks.append(" ".join(chunk))
    
    return chunks

# Application du chunking
df['text'] = df['text'].apply(lambda x: chunk_text(x, chunk_size=200, overlap=50))

### Embeddings des chunks

In [9]:
# Importation et initialisation de Chroma DB
import chromadb
from chromadb.utils import embedding_functions

# Connexion au client Chroma
chroma_client = chromadb.Client()

# Création (ou récupération) d’une collection
collection = chroma_client.get_or_create_collection(name="fake_news")

In [10]:
# Fonction pour génerer les embeddings avec Ollama
import requests

def generate_embeddings(texts, model="all-minilm"):
    """
    Génère les embeddings pour une liste de textes via Ollama.
    Retourne une liste de vecteurs (list of lists).
    """
    embeddings = []

    for text in texts:
        response = requests.post(
            "http://localhost:11434/api/embeddings",
            json={"model": model, "input": text}
        )
        data = response.json()
        embeddings.append(data["embedding"])
    return embeddings


### Géneration des embeddings pour chaque chunks

In [16]:
# Exemple : création d'IDs uniques
df["chunk_id"] = [f"doc_{i}" for i in range(len(df))]
# Création des métadonnées (label et title)
df["metadata"] = df.apply(lambda row: {"title": row["title"],"label": row["label"]},axis=1)
texts = df["text"].tolist()
metadatas = df["metadata"].tolist()

# Génération des embeddings avec Ollama
embeddings = generate_embeddings(texts, model="all-minilm")

## Ajout des embeddings à ChromaDB

In [26]:
# collection.upsert(
#     ids=df["chunk_id"].tolist(),
#     documents=df["text"].tolist(),
#     embeddings=embeddings,
#     metadatas=df["metadata"].tolist()
# )

In [ ]:
# def normalize_vector(vec):
#     norm = np.linalg.norm(vec)
#     return vec / norm if norm > 0 else vec

# df["embeddings"] = df["embeddings"].apply(normalize_vector)
